In [1]:
import pandas as pd
data = pd.read_csv('Project II Data File.csv', header = 1)
data.head()

,Unnamed: 0,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM,Unnamed: 26,RMRF,SMB,HML,Unnamed: 30,RF
0,192607,5.8248,-1.7006,0.4875,-1.4580,2.0534,1.2077,2.4192,0.4926,-2.6049,...,6.0902,2.0266,3.1111,0.5623,NaN,2.96,-2.56,-2.43,NaN,0.22
1,192608,-2.0206,-8.0282,1.3796,1.4606,8.3968,2.3618,-1.1849,4.0084,0.5038,...,4.1903,2.0131,5.4849,7.7576,NaN,2.64,-1.17,3.82,NaN,0.25
2,192609,-4.8291,-2.6154,-4.3417,-3.2729,0.8649,-2.6540,-1.2618,1.0829,-3.5480,...,3.6538,0.0950,-0.7487,-2.4284,NaN,0.36,-1.40,0.13,NaN,0.23
3,192610,-9.3729,-3.5519,-3.4948,3.4413,-2.5476,-2.8069,-3.2663,-5.0745,-8.0191,...,-3.0071,-2.2437,-4.6719,-5.8129,NaN,-3.24,-0.09,0.70,NaN,0.32
4,192611,5.5888,4.1877,2.4623,-4.4494,0.5362,3.1033,-2.3690,3.0078,5.1546,...,2.5326,1.5204,3.6619,2.5636,NaN,2.53,-0.10,-0.51,NaN,0.31


In [2]:
cols = list(data.columns[1:])
date = list(['Date'])
data.columns = date+cols

In [3]:
data.dropna(inplace = True, axis=1)

In [4]:
# data from 1963-07 to 1993-06

data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')  # Convert the 'Date' column to datetime format
data.set_index('Date', inplace=True)


In [5]:
data.head()

,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM,RMRF,SMB,HML,RF
Date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,5.8248,-1.7006,0.4875,-1.4580,2.0534,1.2077,2.4192,0.4926,-2.6049,-0.3344,...,2.4678,3.4539,6.0902,2.0266,3.1111,0.5623,2.96,-2.56,-2.43,0.22
1926-08-01,-2.0206,-8.0282,1.3796,1.4606,8.3968,2.3618,-1.1849,4.0084,0.5038,6.1675,...,5.3422,1.0124,4.1903,2.0131,5.4849,7.7576,2.64,-1.17,3.82,0.25
1926-09-01,-4.8291,-2.6154,-4.3417,-3.2729,0.8649,-2.6540,-1.2618,1.0829,-3.5480,-0.9401,...,0.8730,-1.2906,3.6538,0.0950,-0.7487,-2.4284,0.36,-1.40,0.13,0.23
1926-10-01,-9.3729,-3.5519,-3.4948,3.4413,-2.5476,-2.8069,-3.2663,-5.0745,-8.0191,-1.3213,...,-5.3525,-2.7413,-3.0071,-2.2437,-4.6719,-5.8129,-3.24,-0.09,0.70,0.32
1926-11-01,5.5888,4.1877,2.4623,-4.4494,0.5362,3.1033,-2.3690,3.0078,5.1546,2.7292,...,1.8213,4.2946,2.5326,1.5204,3.6619,2.5636,2.53,-0.10,-0.51,0.31


In [6]:
portfolios = data.columns[:-4]

In [7]:

# Assuming 'data' already has 'Date' as a datetime index
df_fama_time = data.loc['1963-07':'1993-06']
df_modern_time = data.loc['1994-09':'2024-08']


In [8]:
df_fama_time.shape

(360, 29)

# July 1963 to June 1993

## CAPM

In [11]:
# time series regression
import statsmodels.api as sm
import numpy as np 

results = []
residuals = pd.DataFrame(index=df_fama_time.index)  # Create a DataFrame to store residuals


# Iterate through each portfolio and perform regression
for portfolio in portfolios:
    y = df_fama_time[portfolio] - df_fama_time['RF']  # Dependent variable: returns for each portfolio
    x = df_fama_time['RMRF'] # Independent variable: market excess return (RMRF)
    
    # Add a constant (intercept) to the independent variable
    x = sm.add_constant(x)
    
    # Fit the OLS model
    model = sm.OLS(y, x).fit()

    residuals[portfolio] = y - model.fittedvalues  # y - y_pred
    
    # Extract regression results
    alpha = model.params['const']  # Intercept
    beta = model.params['RMRF']    # Coefficient (Beta)
    alpha_se = model.bse['const']  # Standard error of the intercept
    beta_se = model.bse['RMRF']    # Standard error of the beta
    exp_return = np.mean(y)
    
    # Store results in a dictionary
    results.append({
        'Portfolio': portfolio,
        'Alpha': alpha,
        'Alpha SE': alpha_se,
        'Beta': beta,
        'Beta SE': beta_se,
        'Exp_return': exp_return
    })

# Convert results into a DataFrame for better visualization
results_df = pd.DataFrame(results)
results_df.head()

,Portfolio,Alpha,Alpha SE,Beta,Beta SE,Exp_return
0,SMALL LoBM,-0.321399,0.229019,1.422869,0.050748,0.268894
1,ME1 BM2,0.175148,0.198654,1.245335,0.044020,0.691789
2,ME1 BM3,0.278551,0.180111,1.154638,0.039911,0.757565
3,ME1 BM4,0.464942,0.178554,1.072550,0.039566,0.909901
4,SMALL HiBM,0.625586,0.202430,1.101028,0.044857,1.082360


In [12]:
# Correcting T, N, and alpha
T = df_fama_time.shape[0]  # Number of time periods
N = len(portfolios)  # Number of portfolios
K = 1  # One factor in CAPM

# Demean residuals to calculate the covariance matrix
demeaned_res = residuals - np.mean(residuals, axis=0)
err_cov = np.dot(demeaned_res.T, demeaned_res) / (T - 1)

# Extract alphas from the results DataFrame
alpha = np.array(results_df['Alpha'])

# Mean and standard deviation of RMRF
mean_rmrf = np.mean(df_fama_time['RMRF'])
stddev_rmrf = np.std(df_fama_time['RMRF'])

# F-GRS test statistic
f_grs_p1 = ((T - N - K) / N) * (np.dot(np.dot(alpha.T, np.linalg.inv(err_cov)), alpha) / (1 + (mean_rmrf / stddev_rmrf) ** 2))

# Print the F-GRS value
print(f"F-GRS statistic: {f_grs_p1}")

F-GRS statistic: 2.4010416277308804


In [27]:
from scipy.stats import f
f_critical = f.ppf(0.95, T-N-K, N)
if f_grs_p1 > f_critical:
    print ('Alphas are jointly significantly different from zero')
else:
    print ('Alpha are not significantly different from zero')

Alphas are jointly significantly different from zero


In [28]:
#cross sectional regression

y = results_df['Exp_return']
x = results_df['Beta'] # Independent variable: market excess return (RMRF)

# Add a constant (intercept) to the independent variable
x = sm.add_constant(x)

# Fit the OLS model
model = sm.OLS(y, x).fit()

lambda_cs_capm_p1 = model.params['Beta']
lambda_cs_capm_se_p1 = model.bse['Beta']
lambda_tstat_capm_p1 = lambda_cs_capm_p1 / lambda_cs_capm_se_p1
print(f"lambda estimate and t stat for Lambda: {lambda_cs_capm_p1, lambda_tstat_capm_p1}")


lambda estimate and t stat for Lambda: (-0.3710563925095865, -1.131083207251284)


## Fama MacBeth

In [30]:

# List to store regression coefficients (excluding intercepts)
time_step_results = []

# Iterate over each time step (i.e., each row in df_fama_time)
for i in range(df_fama_time.shape[0]):
    # y: Stock returns at time step i (for all portfolios)
    y = np.array(df_fama_time[portfolios].subtract(df_fama_time['RF'], axis=0).iloc[[i]].T)
    
    # x: Beta values from results_df (previously calculated coefficients)
    x = np.array(results_df[['Beta']])  # Use previously calculated Betas for each portfolio
    
    # Add constant (intercept) to the independent variable
    x = sm.add_constant(x)
    
    # Fit OLS regression: y = alpha + beta * x
    model = sm.OLS(y, x).fit()
    
    # Store the coefficient (excluding intercept)
    time_step_results.append(model.params[1:])  # model.params[1:] excludes the intercept

 
# Convert the results into a DataFrame for better visualization
time_step_results_df = pd.DataFrame(time_step_results, index=df_fama_time.index, columns=['lambda'])


In [31]:
mean_lambda_capm_p1_fmp = np.mean(time_step_results_df['lambda'])
std_dev_lambda_capm_p1_fmp = np.std(time_step_results_df['lambda'])/np.sqrt(T)
lambda_tstat_capm_p1_fmp = mean_lambda_capm_p1_fmp/std_dev_lambda_capm_p1_fmp
print(f"lambda estimate and t stat for Lambda: {mean_lambda_capm_p1_fmp, lambda_tstat_capm_p1_fmp}")

lambda estimate and t stat for Lambda: (-0.3710563925095867, -0.8056834710949038)


## Fama French 3 Factor

In [33]:

results = []
residuals = pd.DataFrame(index=df_fama_time.index)  # Create a DataFrame to store residuals

# Iterate through each portfolio and perform regression
for portfolio in portfolios:
    y = df_fama_time[portfolio] - df_fama_time['RF']  # Dependent variable: excess returns for each portfolio (portfolio returns - risk-free rate)
    
    # Independent variables: Fama-French factors (RMRF, SMB, HML)
    x = df_fama_time[['RMRF', 'SMB', 'HML']]  
    
    # Add a constant (intercept) to the independent variables
    x = sm.add_constant(x)
    
    # Fit the OLS model
    model = sm.OLS(y, x).fit()

    # Calculate residuals (y - y_pred)
    residuals[portfolio] = y - model.fittedvalues  
    
    # Extract regression results
    alpha = model.params['const']  # Intercept (alpha)
    beta_rmrf = model.params['RMRF']  # Coefficient for RMRF
    beta_smb = model.params['SMB']    # Coefficient for SMB
    beta_hml = model.params['HML']    # Coefficient for HML
    alpha_se = model.bse['const']     # Standard error of the intercept
    beta_rmrf_se = model.bse['RMRF']  # Standard error of the RMRF coefficient
    beta_smb_se = model.bse['SMB']    # Standard error of the SMB coefficient
    beta_hml_se = model.bse['HML']    # Standard error of the HML coefficient
    exp_return = np.mean(y)           # Expected excess return
    
    # Store results in a dictionary
    results.append({
        'Portfolio': portfolio,
        'Alpha': alpha,
        'Alpha SE': alpha_se,
        'Beta RMRF': beta_rmrf,
        'Beta RMRF SE': beta_rmrf_se,
        'Beta SMB': beta_smb,
        'Beta SMB SE': beta_smb_se,
        'Beta HML': beta_hml,
        'Beta HML SE': beta_hml_se,
        'Exp_return': exp_return
    })

# Convert results into a DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display the first few rows of the results
results_df.head()

,Portfolio,Alpha,Alpha SE,Beta RMRF,Beta RMRF SE,Beta SMB,Beta SMB SE,Beta HML,Beta HML SE,Exp_return
0,SMALL LoBM,-0.410215,0.104763,1.030324,0.025892,1.427296,0.037924,-0.291606,0.042023,0.268894
1,ME1 BM2,-0.101100,0.078674,0.966742,0.019444,1.282514,0.028480,0.096255,0.031558,0.691789
2,ME1 BM3,-0.069238,0.063867,0.941189,0.015784,1.154731,0.023120,0.267346,0.025619,0.757565
3,ME1 BM4,0.051601,0.060324,0.895363,0.014909,1.116126,0.021837,0.399173,0.024197,0.909901
4,SMALL HiBM,0.067140,0.065961,0.953588,0.016302,1.199920,0.023878,0.637196,0.026459,1.082360


In [34]:
# Correcting T, N, and alpha
T = df_fama_time.shape[0]  # Number of time periods
N = len(portfolios)  # Number of portfolios
K = 3  

# Demean residuals to calculate the covariance matrix
demeaned_res = residuals - np.mean(residuals, axis=0)
err_cov = np.dot(demeaned_res.T, demeaned_res) / (T - 1)

factor_mu_vec = np.array(df_fama_time[['RMRF', 'SMB', 'HML']])
demeaned_fac_mu = factor_mu_vec - np.mean(factor_mu_vec, axis=0)
fac_mu_cov = np.dot(demeaned_fac_mu.T, demeaned_fac_mu) / (T - 1)

# Extract alphas from the results DataFrame
alpha = np.array(results_df['Alpha'])

# F-GRS test statistic
f_grs_ff3_p1 = ((T - N - K) / N) * (np.dot(np.dot(alpha.T, np.linalg.inv(err_cov)), alpha) / (1 + np.dot(np.dot(np.mean(factor_mu_vec,axis=0), np.linalg.inv(fac_mu_cov)), np.mean(factor_mu_vec,axis=0).T)))

# Print the F-GRS value
print(f"F-GRS statistic: {f_grs_ff3_p1}")

F-GRS statistic: 1.6233947742528965


In [35]:
f_grs_ff3_p1

1.6233947742528965

In [36]:

f_critical = f.ppf(0.95, T-N-K, N)
if f_grs_ff3_p1 > f_critical:
    print ('Alphas are jointly significantly different from zero')
else:
    print ('Alpha are not significantly different from zero')

Alpha are not significantly different from zero


In [37]:


# Cross-sectional regression

# y: Expected returns
y = results_df['Exp_return']

# x: Independent variables (Beta RMRF, Beta SMB, Beta HML)
x = results_df[['Beta RMRF', 'Beta SMB', 'Beta HML']]

# Add a constant (intercept) to the independent variables
x = sm.add_constant(x)

# Fit the OLS model
model = sm.OLS(y, x).fit()

# Store the coefficients (lambdas) and their standard errors
lambda_cs_ff3_p1 = model.params[['Beta RMRF', 'Beta SMB', 'Beta HML']]  # Coefficients for RMRF, SMB, HML
lambda_cs_ff3_se_p1 = model.bse[['Beta RMRF', 'Beta SMB', 'Beta HML']]  # Standard errors for RMRF, SMB, HML

# Calculate t-statistics for the coefficients (lambda estimates)
lambda_tstat_ff3_p1 = lambda_cs_ff3_p1 / lambda_cs_ff3_se_p1

# Print results
print(f"Lambda estimates: \n{lambda_cs_ff3_p1}")
print(f"\nStandard Errors: \n{lambda_cs_ff3_se_p1}")
print(f"\nT-statistics for Lambda: \n{lambda_tstat_ff3_p1}")

Lambda estimates: 
Beta RMRF   -0.096158
Beta SMB     0.200254
Beta HML     0.513732
dtype: float64

Standard Errors: 
Beta RMRF    0.446402
Beta SMB     0.052182
Beta HML     0.065251
dtype: float64

T-statistics for Lambda: 
Beta RMRF   -0.215407
Beta SMB     3.837592
Beta HML     7.873120
dtype: float64


## Fama MacBeth

In [39]:

# List to store regression coefficients (excluding intercepts)
time_step_results = []

# Iterate over each time step (i.e., each row in df_fama_time)
for i in range(df_fama_time.shape[0]):
    # y: Stock returns at time step i (for all portfolios)
    y = np.array(df_fama_time[portfolios].subtract(df_fama_time['RF'], axis=0).iloc[[i]].T)
    
    # x: Beta values from results_df (previously calculated coefficients)
    x = np.array(results_df[['Beta RMRF', 'Beta SMB', 'Beta HML']])  # Use previously calculated Betas for each portfolio
    
    # Add constant (intercept) to the independent variable
    x = sm.add_constant(x)
    
    # Fit OLS regression: y = alpha + beta * x
    model = sm.OLS(y, x).fit()
    
    # Store the coefficient (excluding intercept)
    time_step_results.append(model.params[1:])  # model.params[1:] excludes the intercept

 
# Convert the results into a DataFrame for better visualization
time_step_results_df = pd.DataFrame(time_step_results, index=df_fama_time.index, columns=['lambda RMRF','lambda SMB','lambda HML'])



In [40]:
lambda_cs_ff3_fmp_p1 = np.mean(time_step_results_df, axis=0)
lambda_cs_ff3_se_fmp_p1 = np.std(time_step_results_df, axis=0)/np.sqrt(T)

# Calculate t-statistics for the coefficients (lambda estimates)
lambda_tstat_ff3_fmp_p1 = lambda_cs_ff3_fmp_p1 / lambda_cs_ff3_se_fmp_p1

# Print results
print(f"Lambda estimates: \n{lambda_cs_ff3_fmp_p1}")
print(f"\nStandard Errors: \n{lambda_cs_ff3_se_fmp_p1}")
print(f"\nT-statistics for Lambda: \n{lambda_tstat_ff3_fmp_p1}")

Lambda estimates: 
lambda RMRF   -0.096158
lambda SMB     0.200254
lambda HML     0.513732
dtype: float64

Standard Errors: 
lambda RMRF    0.428920
lambda SMB     0.154322
lambda HML     0.139354
dtype: float64

T-statistics for Lambda: 
lambda RMRF   -0.224187
lambda SMB     1.297634
lambda HML     3.686513
dtype: float64


# Sep 1994 to August 2024

## CAPM

In [43]:
# time series regression
import statsmodels.api as sm
import numpy as np 

results = []
residuals = pd.DataFrame(index=df_modern_time.index)  # Create a DataFrame to store residuals


# Iterate through each portfolio and perform regression
for portfolio in portfolios:
    y = df_modern_time[portfolio] - df_modern_time['RF']  # Dependent variable: returns for each portfolio
    x = df_modern_time['RMRF'] # Independent variable: market excess return (RMRF)
    
    # Add a constant (intercept) to the independent variable
    x = sm.add_constant(x)
    
    # Fit the OLS model
    model = sm.OLS(y, x).fit()

    residuals[portfolio] = y - model.fittedvalues  # y - y_pred
    
    # Extract regression results
    alpha = model.params['const']  # Intercept
    beta = model.params['RMRF']    # Coefficient (Beta)
    alpha_se = model.bse['const']  # Standard error of the intercept
    beta_se = model.bse['RMRF']    # Standard error of the beta
    exp_return = np.mean(y)
    
    # Store results in a dictionary
    results.append({
        'Portfolio': portfolio,
        'Alpha': alpha,
        'Alpha SE': alpha_se,
        'Beta': beta,
        'Beta SE': beta_se,
        'Exp_return': exp_return
    })

# Convert results into a DataFrame for better visualization
results_df = pd.DataFrame(results)
results_df.head()

,Portfolio,Alpha,Alpha SE,Beta,Beta SE,Exp_return
0,SMALL LoBM,-0.733069,0.295515,1.411599,0.064488,0.333433
1,ME1 BM2,-0.061568,0.259056,1.207030,0.056532,0.850376
2,ME1 BM3,-0.010913,0.197824,1.074166,0.043169,0.800649
3,ME1 BM4,0.264413,0.204839,1.009707,0.044700,1.027275
4,SMALL HiBM,0.334630,0.239200,1.046573,0.052199,1.125344


In [44]:
# Correcting T, N, and alpha
T = df_modern_time.shape[0]  # Number of time periods
N = len(portfolios)  # Number of portfolios
K = 1  # One factor in CAPM

# Demean residuals to calculate the covariance matrix
demeaned_res = residuals - np.mean(residuals, axis=0)
err_cov = np.dot(demeaned_res.T, demeaned_res) / (T - 1)

# Extract alphas from the results DataFrame
alpha = np.array(results_df['Alpha'])

# Mean and standard deviation of RMRF
mean_rmrf = np.mean(df_modern_time['RMRF'])
stddev_rmrf = np.std(df_modern_time['RMRF'])

# F-GRS test statistic
f_grs_capm_p2 = ((T - N - K) / N) * (np.dot(np.dot(alpha.T, np.linalg.inv(err_cov)), alpha) / (1 + (mean_rmrf / stddev_rmrf) ** 2))

# Print the F-GRS value
print(f"F-GRS statistic: {f_grs_capm_p2}")

F-GRS statistic: 3.8630854740794662


In [45]:
from scipy.stats import f
f_critical = f.ppf(0.95, T-N-K, N)
if f_grs_capm_p2 > f_critical:
    print ('Alphas are jointly significantly different from zero')
else:
    print ('Alpha are not significantly different from zero')

Alphas are jointly significantly different from zero


In [46]:
#cross sectional regression

y = results_df['Exp_return']
x = results_df['Beta'] # Independent variable: market excess return (RMRF)

# Add a constant (intercept) to the independent variable
x = sm.add_constant(x)

# Fit the OLS model
model = sm.OLS(y, x).fit()

lambda_cs_capm_p2 = model.params['Beta']
lambda_cs_capm_se_p2 = model.bse['Beta']
lambda_tstat_capm_p2 = lambda_cs_capm_p2 / lambda_cs_capm_se_p2
print(f"lambda estimate and t stat for Lambda: {lambda_cs_capm_p2, lambda_tstat_capm_p2}")

lambda estimate and t stat for Lambda: (-0.4387947188334112, -1.9110617288246792)


## Fama MacBeth

In [48]:

# List to store regression coefficients (excluding intercepts)
time_step_results = []

# Iterate over each time step (i.e., each row in df_fama_time)
for i in range(df_modern_time.shape[0]):
    # y: Stock returns at time step i (for all portfolios)
    y = np.array(df_modern_time[portfolios].subtract(df_modern_time['RF'], axis=0).iloc[[i]].T)
    
    # x: Beta values from results_df (previously calculated coefficients)
    x = np.array(results_df[['Beta']])  # Use previously calculated Betas for each portfolio
    
    # Add constant (intercept) to the independent variable
    x = sm.add_constant(x)
    
    # Fit OLS regression: y = alpha + beta * x
    model = sm.OLS(y, x).fit()
    
    # Store the coefficient (excluding intercept)
    time_step_results.append(model.params[1:])  # model.params[1:] excludes the intercept

 
# Convert the results into a DataFrame for better visualization
time_step_results_df = pd.DataFrame(time_step_results, index=df_modern_time.index, columns=['lambda'])


In [49]:
mean_lambda_capm_fmp_p2 = np.mean(time_step_results_df['lambda'])
std_dev_lambda_capm_fmp_p2 = np.std(time_step_results_df['lambda'])/np.sqrt(T)
lambda_tstat_capm_fmp_p2 = mean_lambda_capm_fmp_p2/std_dev_lambda_capm_fmp_p2
print(f"lambda estimate and t stat for Lambda: {mean_lambda_capm_fmp_p2, lambda_tstat_capm_fmp_p2}")

lambda estimate and t stat for Lambda: (-0.43879471883341115, -0.6865568531715411)


## Fama French 3 Factor

In [51]:

results = []
residuals = pd.DataFrame(index=df_modern_time.index)  # Create a DataFrame to store residuals

# Iterate through each portfolio and perform regression
for portfolio in portfolios:
    y = df_modern_time[portfolio] - df_modern_time['RF']  # Dependent variable: excess returns for each portfolio (portfolio returns - risk-free rate)
    
    # Independent variables: Fama-French factors (RMRF, SMB, HML)
    x = df_modern_time[['RMRF', 'SMB', 'HML']]  
    
    # Add a constant (intercept) to the independent variables
    x = sm.add_constant(x)
    
    # Fit the OLS model
    model = sm.OLS(y, x).fit()

    # Calculate residuals (y - y_pred)
    residuals[portfolio] = y - model.fittedvalues  
    
    # Extract regression results
    alpha = model.params['const']  # Intercept (alpha)
    beta_rmrf = model.params['RMRF']  # Coefficient for RMRF
    beta_smb = model.params['SMB']    # Coefficient for SMB
    beta_hml = model.params['HML']    # Coefficient for HML
    alpha_se = model.bse['const']     # Standard error of the intercept
    beta_rmrf_se = model.bse['RMRF']  # Standard error of the RMRF coefficient
    beta_smb_se = model.bse['SMB']    # Standard error of the SMB coefficient
    beta_hml_se = model.bse['HML']    # Standard error of the HML coefficient
    exp_return = np.mean(y)           # Expected excess return
    
    # Store results in a dictionary
    results.append({
        'Portfolio': portfolio,
        'Alpha': alpha,
        'Alpha SE': alpha_se,
        'Beta RMRF': beta_rmrf,
        'Beta RMRF SE': beta_rmrf_se,
        'Beta SMB': beta_smb,
        'Beta SMB SE': beta_smb_se,
        'Beta HML': beta_hml,
        'Beta HML SE': beta_hml_se,
        'Exp_return': exp_return
    })

# Convert results into a DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display the first few rows of the results
results_df.head()

,Portfolio,Alpha,Alpha SE,Beta RMRF,Beta RMRF SE,Beta SMB,Beta SMB SE,Beta HML,Beta HML SE,Exp_return
0,SMALL LoBM,-0.582840,0.155178,1.150553,0.034871,1.410517,0.049049,-0.267979,0.046058,0.333433
1,ME1 BM2,0.046585,0.120208,0.971645,0.027013,1.348594,0.037996,-0.034767,0.035679,0.850376
2,ME1 BM3,0.021005,0.084282,0.910023,0.018940,1.062648,0.026640,0.305280,0.025016,0.800649
3,ME1 BM4,0.261375,0.077027,0.858452,0.017309,1.070382,0.024347,0.527118,0.022862,1.027275
4,SMALL HiBM,0.297880,0.129288,0.912353,0.029053,1.045504,0.040866,0.725705,0.038374,1.125344


In [52]:
# Correcting T, N, and alpha
T = df_modern_time.shape[0]  # Number of time periods
N = len(portfolios)  # Number of portfolios
K = 3  

# Demean residuals to calculate the covariance matrix
demeaned_res = residuals - np.mean(residuals, axis=0)
err_cov = np.dot(demeaned_res.T, demeaned_res) / (T - 1)

factor_mu_vec = np.array(df_modern_time[['RMRF', 'SMB', 'HML']])
demeaned_fac_mu = factor_mu_vec - np.mean(factor_mu_vec, axis=0)
fac_mu_cov = np.dot(demeaned_fac_mu.T, demeaned_fac_mu) / (T - 1)

# Extract alphas from the results DataFrame
alpha = np.array(results_df['Alpha'])

# F-GRS test statistic
f_grs_ff3_p2 = ((T - N - K) / N) * (np.dot(np.dot(alpha.T, np.linalg.inv(err_cov)), alpha) / (1 + np.dot(np.dot(np.mean(factor_mu_vec,axis=0), np.linalg.inv(fac_mu_cov)), np.mean(factor_mu_vec,axis=0).T)))

# Print the F-GRS value
print(f"F-GRS statistic: {f_grs_ff3_p2}")

F-GRS statistic: 3.8543498028529366


In [54]:

f_critical = f.ppf(0.95, T-N-K, N)
if f_grs_ff3_p2 > f_critical:
    print ('Alphas are jointly significantly different from zero')
else:
    print ('Alpha are not significantly different from zero')

Alphas are jointly significantly different from zero


In [55]:


# Cross-sectional regression

# y: Expected returns
y = results_df['Exp_return']

# x: Independent variables (Beta RMRF, Beta SMB, Beta HML)
x = results_df[['Beta RMRF', 'Beta SMB', 'Beta HML']]

# Add a constant (intercept) to the independent variables
x = sm.add_constant(x)

# Fit the OLS model
model = sm.OLS(y, x).fit()

# Store the coefficients (lambdas) and their standard errors
lambda_cs_ff3_p2 = model.params[['Beta RMRF', 'Beta SMB', 'Beta HML']]  # Coefficients for RMRF, SMB, HML
lambda_cs_ff3_se_p2 = model.bse[['Beta RMRF', 'Beta SMB', 'Beta HML']]  # Standard errors for RMRF, SMB, HML

# Calculate t-statistics for the coefficients (lambda estimates)
lambda_tstat_ff3_p2 = lambda_cs_ff3_p2 / lambda_cs_ff3_se_p2

# Print results
print(f"Lambda estimates: \n{lambda_cs_ff3_p2}")
print(f"\nStandard Errors: \n{lambda_cs_ff3_se_p2}")
print(f"\nT-statistics for Lambda: \n{lambda_tstat_ff3_p2}")

Lambda estimates: 
Beta RMRF   -0.781531
Beta SMB     0.030811
Beta HML     0.133704
dtype: float64

Standard Errors: 
Beta RMRF    0.319718
Beta SMB     0.055714
Beta HML     0.068633
dtype: float64

T-statistics for Lambda: 
Beta RMRF   -2.444440
Beta SMB     0.553023
Beta HML     1.948116
dtype: float64


## Fama MacBeth

In [57]:

# List to store regression coefficients (excluding intercepts)
time_step_results = []

# Iterate over each time step (i.e., each row in df_fama_time)
for i in range(df_fama_time.shape[0]):
    # y: Stock returns at time step i (for all portfolios)
    y = np.array(df_modern_time[portfolios].subtract(df_modern_time['RF'], axis=0).iloc[[i]].T)
    
    # x: Beta values from results_df (previously calculated coefficients)
    x = np.array(results_df[['Beta RMRF', 'Beta SMB', 'Beta HML']])  # Use previously calculated Betas for each portfolio
    
    # Add constant (intercept) to the independent variable
    x = sm.add_constant(x)
    
    # Fit OLS regression: y = alpha + beta * x
    model = sm.OLS(y, x).fit()
    
    # Store the coefficient (excluding intercept)
    time_step_results.append(model.params[1:])  # model.params[1:] excludes the intercept

 
# Convert the results into a DataFrame for better visualization
time_step_results_df = pd.DataFrame(time_step_results, index=df_modern_time.index, columns=['lambda RMRF','lambda SMB','lambda HML'])



In [58]:
lambda_cs_ff3_fmp_p2 = np.mean(time_step_results_df, axis=0)
lambda_cs_ff3_se_fmp_p2 = np.std(time_step_results_df, axis=0)/np.sqrt(T)

# Calculate t-statistics for the coefficients (lambda estimates)
lambda_tstat_ff3_fmp_p2 = lambda_cs_ff3_fmp_p2 / lambda_cs_ff3_se_fmp_p2

# Print results
print(f"Lambda estimates: \n{lambda_cs_ff3_fmp_p2}")
print(f"\nStandard Errors: \n{lambda_cs_ff3_se_fmp_p2}")
print(f"\nT-statistics for Lambda: \n{lambda_tstat_ff3_fmp_p2}")

Lambda estimates: 
lambda RMRF   -0.781531
lambda SMB     0.030811
lambda HML     0.133704
dtype: float64

Standard Errors: 
lambda RMRF    0.423370
lambda SMB     0.177102
lambda HML     0.182601
dtype: float64

T-statistics for Lambda: 
lambda RMRF   -1.845976
lambda SMB     0.173974
lambda HML     0.732220
dtype: float64


# Aggregated Results

In [60]:


# Combine all the lists into one for easier DataFrame creation
data = {
    "Time Period": ["7/63 – 6/93", "9/94 – 8/24"],
    "Lambda (β)": [lambda_cs_capm_p1, lambda_cs_capm_p2],
    "Standard Error": [lambda_cs_capm_se_p1, lambda_cs_capm_se_p2],
    "T-Statistic": [lambda_tstat_capm_p1, lambda_tstat_capm_p2],
    "Significant?": ["Yes" if abs(lambda_tstat_capm_p1) > 1.96 else "No", 
                     "Yes" if abs(lambda_tstat_capm_p2) > 1.96 else "No"]
}

# Create the DataFrame
df1 = pd.DataFrame(data)
df1.index = df1['Time Period']
np.round(df1,2)


,Time Period,Lambda (β),Standard Error,T-Statistic,Significant?
Time Period,,,,,
7/63 – 6/93,7/63 – 6/93,-0.37,0.33,-1.13,No
9/94 – 8/24,9/94 – 8/24,-0.44,0.23,-1.91,No


In [61]:


# Combine all the lists into one for easier DataFrame creation
data = {
    "Time Period": ["7/63 – 6/93", "9/94 – 8/24"],
    "Lambda (β)": [mean_lambda_capm_p1_fmp, mean_lambda_capm_fmp_p2],
    "Standard Error": [std_dev_lambda_capm_p1_fmp, std_dev_lambda_capm_fmp_p2],
    "T-Statistic": [lambda_tstat_capm_p1_fmp, lambda_tstat_capm_fmp_p2],
    "Significant?": ["Yes" if abs(mean_lambda_capm_p1_fmp) > 1.96 else "No", 
                     "Yes" if abs(lambda_tstat_capm_fmp_p2) > 1.96 else "No"]
}

# Create the DataFrame
df2 = pd.DataFrame(data)
df2.index = df2['Time Period']
np.round(df2,2)



,Time Period,Lambda (β),Standard Error,T-Statistic,Significant?
Time Period,,,,,
7/63 – 6/93,7/63 – 6/93,-0.37,0.46,-0.81,No
9/94 – 8/24,9/94 – 8/24,-0.44,0.64,-0.69,No


In [62]:
f_crit_capm = f.ppf(0.95,  N, T-N-1)

# Combine all the lists into one for easier DataFrame creation
data = {
    "Time Period": ["7/63 – 6/93", "9/94 – 8/24"],
    "f_grs": [f_grs_p1, f_grs_capm_p2],
    "Critical Value": [f_crit_capm, f_crit_capm],
    "Significant?": ["Yes" if abs(f_grs_ff3_p1) > f_crit_capm else "No", 
                     "Yes" if abs(f_grs_ff3_p2) > f_crit_capm else "No"]
}

# Create the DataFrame
df3 = pd.DataFrame(data)
df3.index = df3['Time Period']
np.round(df3,2)


,Time Period,f_grs,Critical Value,Significant?
Time Period,,,,
7/63 – 6/93,7/63 – 6/93,2.40,1.54,Yes
9/94 – 8/24,9/94 – 8/24,3.86,1.54,Yes


In [63]:

# Create the DataFrame with the relevant data
data = {
    "Time Period": ["7/63 – 6/93", "9/94 – 8/24"],
    "λ_b": [lambda_cs_ff3_p1.values[0], lambda_cs_ff3_p2.values[0]],
    "σ(λ_b)": [lambda_cs_ff3_se_p1.values[0], lambda_cs_ff3_se_p2.values[0]],
    "t(λ_b)": [lambda_tstat_ff3_p1.values[0], lambda_tstat_ff3_p2.values[0]],
    "λ_s": [lambda_cs_ff3_p1.values[1], lambda_cs_ff3_p2.values[1]],
    "σ(λ_s)": [lambda_cs_ff3_se_p1.values[1], lambda_cs_ff3_se_p2.values[1]],
    "t(λ_s)": [lambda_tstat_ff3_p1.values[1], lambda_tstat_ff3_p2.values[1]],
    "λ_h": [lambda_cs_ff3_p1.values[2], lambda_cs_ff3_p2.values[2]],
    "σ(λ_h)": [lambda_cs_ff3_se_p1.values[2], lambda_cs_ff3_se_p2.values[2]],
    "t(λ_h)": [lambda_tstat_ff3_p1.values[2], lambda_tstat_ff3_p2.values[2]]
}

# Create the DataFrame
df4 = pd.DataFrame(data)
df4.index = df4['Time Period']
np.round(df4,2)

,Time Period,λ_b,σ(λ_b),t(λ_b),λ_s,σ(λ_s),t(λ_s),λ_h,σ(λ_h),t(λ_h)
Time Period,,,,,,,,,,
7/63 – 6/93,7/63 – 6/93,-0.10,0.45,-0.22,0.20,0.05,3.84,0.51,0.07,7.87
9/94 – 8/24,9/94 – 8/24,-0.78,0.32,-2.44,0.03,0.06,0.55,0.13,0.07,1.95


In [64]:

# Create the DataFrame with the relevant data
data = {
    "Time Period": ["7/63 – 6/93", "9/94 – 8/24"],
    "λ_b": [lambda_cs_ff3_fmp_p1.values[0], lambda_cs_ff3_fmp_p2.values[0]],
    "σ(λ_b)": [lambda_cs_ff3_se_fmp_p1.values[0], lambda_cs_ff3_se_fmp_p2.values[0]],
    "t(λ_b)": [lambda_tstat_ff3_fmp_p1.values[0], lambda_tstat_ff3_fmp_p2.values[0]],
    "λ_s": [lambda_cs_ff3_fmp_p1.values[1], lambda_cs_ff3_fmp_p2.values[1]],
    "σ(λ_s)": [lambda_cs_ff3_se_fmp_p1.values[1], lambda_cs_ff3_se_fmp_p2.values[1]],
    "t(λ_s)": [lambda_tstat_ff3_fmp_p1.values[1], lambda_tstat_ff3_fmp_p2.values[1]],
    "λ_h": [lambda_cs_ff3_fmp_p1.values[2], lambda_cs_ff3_fmp_p2.values[2]],
    "σ(λ_h)": [lambda_cs_ff3_se_fmp_p1.values[2], lambda_cs_ff3_se_fmp_p2.values[2]],
    "t(λ_h)": [lambda_tstat_ff3_fmp_p1.values[2], lambda_tstat_ff3_fmp_p2.values[2]]
}

# Create the DataFrame
df5 = pd.DataFrame(data)
df5.index = df5['Time Period']
np.round(df5,2)

,Time Period,λ_b,σ(λ_b),t(λ_b),λ_s,σ(λ_s),t(λ_s),λ_h,σ(λ_h),t(λ_h)
Time Period,,,,,,,,,,
7/63 – 6/93,7/63 – 6/93,-0.10,0.43,-0.22,0.20,0.15,1.30,0.51,0.14,3.69
9/94 – 8/24,9/94 – 8/24,-0.78,0.42,-1.85,0.03,0.18,0.17,0.13,0.18,0.73


In [65]:
f_crit_capm = f.ppf(0.95, N,T-N-3)

# Combine all the lists into one for easier DataFrame creation
data = {
    "Time Period": ["7/63 – 6/93", "9/94 – 8/24"],
    "f_grs": [f_grs_ff3_p1, f_grs_ff3_p2],
    "Critical Value": [f_crit_capm, f_crit_capm],
    "Significant?": ["Yes" if abs(f_grs_ff3_p1) > f_crit_capm else "No", 
                     "Yes" if abs(f_grs_ff3_p2) > f_crit_capm else "No"]
}

# Create the DataFrame
df6 = pd.DataFrame(data)
df6.index = df6['Time Period']
np.round(df6,2)


,Time Period,f_grs,Critical Value,Significant?
Time Period,,,,
7/63 – 6/93,7/63 – 6/93,1.62,1.54,Yes
9/94 – 8/24,9/94 – 8/24,3.85,1.54,Yes
